In [1]:
import numpy as np
import xgboost as xgb
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
import optuna
import pickle
from datetime import timedelta
import time

In [2]:
feat = pd.read_csv('./data/test.csv')
FEATURE_COLS = feat.columns[1:].tolist()

In [3]:
# pickle_file_path = './data/test_df.pickle'

# with open(pickle_file_path, 'rb') as f:
#     test_df = pickle.load(f)

pickle_file_path = './data/train_testi_df.pickle'

with open(pickle_file_path, 'rb') as f:
    train_df = pickle.load(f)

In [4]:
testi = train_df[:2000]
train_df = train_df[2000:]

In [5]:
def get_combined_data(df):
    # Oletetaan, että FEATURES_COLS on jo määritelty olemassa oleville piirteille
    data = [df[col].values for col in FEATURE_COLS]
    # Lisää mallin piirteet
    data.append(np.vstack(df['combined_features'].values))
    return np.column_stack(data)

def objective(trial, df, target):
    param = {        
        'objective': 'reg:squarederror',        
        'device' : 'cuda',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log = True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log = True),
        'colsample_bytree': trial.suggest_categorical('colsample_bytree', [0.3, 0.5, 0.7, 1.0]),
        'subsample': trial.suggest_categorical('subsample', [0.6, 0.8, 1.0]),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01, 0.02, 0.05]),
        'n_estimators': trial.suggest_categorical('n_estimators', [100, 200, 150, 300]),
        'max_depth': trial.suggest_categorical('max_depth', [2, 3, 5, 7, 9]),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 100),
        'early_stopping_rounds': 10  }
    
    mse_scores = []

    for fold in df['fold'].unique():
        train_data = df[df['fold'] != fold]
        valid_data = df[df['fold'] == fold]

        X_train = get_combined_data(train_data)
        X_valid = get_combined_data(valid_data)

        y_train = train_data[target]
        y_valid = valid_data[target]

    
        model = xgb.XGBRegressor(**param)
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=0)
        preds = model.predict(X_valid)
        mse = mean_squared_error(y_valid, preds)
        mse_scores.append(mse)

    average_mse = np.mean(mse_scores)
    return average_mse

def optimize_model(df, target):

    start_time = time.time()
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, df, target), n_trials=15)
    print(f'Optimization finished in {timedelta(seconds=time.time() - start_time)}')

    best_params = study.best_trial.params    
    if 'early_stopping_rounds' in best_params:
        del best_params['early_stopping_rounds']
    print(f"Best parameters for {target}: ", best_params)
    
    # Käytä parhaita parametreja koko datan kouluttamiseen
    X = get_combined_data(df)
    y = df[target]
    model = xgb.XGBRegressor(**best_params)
    model.fit(X, y)
    
    return model



In [6]:
def prepare_features(row):
    return np.array(row[f'model_features_423_std_powerlog_3_finetuned'])

train_df['combined_features'] = train_df.apply(prepare_features, axis=1)
# test_df['combined_features'] = test_df.apply(prepare_features, axis=1)
testi['combined_features'] = testi.apply(prepare_features, axis=1)


In [7]:
target_columns = ['X4_mean', 'X11_mean', 'X18_mean', 'X50_mean', 'X26_mean', 'X3112_mean']

# Mallien kouluttaminen jokaiselle kohdemuuttujalle
models = {}
for target in target_columns:    
    print(f'\n\nOptimizing model for {target}\n\n')
    models[target] = optimize_model(train_df, target)

[I 2024-04-25 17:34:14,149] A new study created in memory with name: no-name-7d0ecf96-8a8e-4ff6-aaa0-f54ae723be50




Optimizing model for X4_mean




/home/tobias/miniconda3/envs/tf/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [17:34:21] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1705650282415/work/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)
[I 2024-04-25 17:34:47,186] Trial 0 finished with value: 0.014767306798050186 and parameters: {'lambda': 4.45328572879446e-06, 'alpha': 4.8413489204266845e-06, 'colsample_bytree': 1.0, 'subsample': 0.8, 'learning_rate': 0.05, 'n_estimators': 100, 'max_depth': 9, 'min_child_weight': 56}. Best is trial 0 with value: 0.014767306798050186.
[I 2024-04-25 17:35:0

Optimization finished in 0:02:34.224883
Best parameters for X4_mean:  {'lambda': 4.45328572879446e-06, 'alpha': 4.8413489204266845e-06, 'colsample_bytree': 1.0, 'subsample': 0.8, 'learning_rate': 0.05, 'n_estimators': 100, 'max_depth': 9, 'min_child_weight': 56}


[I 2024-04-25 17:37:15,979] A new study created in memory with name: no-name-b9bc0e33-3669-4dc1-90ed-99bc3758510f


...

Optimization finished in 0:02:38.566113
Best parameters for X26_mean:  {'lambda': 1.89937059558303e-08, 'alpha': 0.03826896768772912, 'colsample_bytree': 0.5, 'subsample': 0.6, 'learning_rate': 0.05, 'n_estimators': 100, 'max_depth': 7, 'min_child_weight': 54}


[I 2024-04-25 17:51:55,509] A new study created in memory with name: no-name-fecefb42-7d24-4b86-89f3-f7eaf1dccd56




Optimizing model for X3112_mean




[I 2024-04-25 17:52:09,480] Trial 0 finished with value: 4038742.4890485955 and parameters: {'lambda': 0.011651712715905014, 'alpha': 1.9834579040302317e-08, 'colsample_bytree': 1.0, 'subsample': 0.6, 'learning_rate': 0.02, 'n_estimators': 100, 'max_depth': 2, 'min_child_weight': 80}. Best is trial 0 with value: 4038742.4890485955.
[I 2024-04-25 17:53:01,823] Trial 1 finished with value: 3742098.718862331 and parameters: {'lambda': 0.28525264165915365, 'alpha': 1.8531908483751415e-06, 'colsample_bytree': 0.7, 'subsample': 0.8, 'learning_rate': 0.02, 'n_estimators': 150, 'max_depth': 7, 'min_child_weight': 58}. Best is trial 1 with value: 3742098.718862331.
[I 2024-04-25 17:53:23,876] Trial 2 finished with value: 3850206.8316072794 and parameters: {'lambda': 0.0013324611247932676, 'alpha': 1.0604149868308294e-06, 'colsample_bytree': 0.5, 'subsample': 1.0, 'learning_rate': 0.02, 'n_estimators': 100, 'max_depth': 5, 'min_child_weight': 19}. Best is trial 1 with value: 3742098.718862331.
[

Optimization finished in 0:03:32.459077
Best parameters for X3112_mean:  {'lambda': 0.02794196288107175, 'alpha': 1.363677972637457e-06, 'colsample_bytree': 0.5, 'subsample': 1.0, 'learning_rate': 0.01, 'n_estimators': 300, 'max_depth': 7, 'min_child_weight': 41}


In [8]:
features_array = np.array(testi['combined_features'].tolist())
X_combined_train = np.hstack([testi[FEATURE_COLS].values, features_array])

train_pred = np.zeros((testi.shape[0], len(target_columns)))

for i, target in enumerate(target_columns):    
    train_pred[:, i] = models[target].predict(X_combined_train)

train_r2 = r2_score(testi[target_columns], train_pred)
print(f'Training R2: {train_r2}')

Training R2: 0.2211687070920861


In [9]:
features_array = np.array(testi['combined_features'].tolist())
X_combined_test = np.hstack([testi[FEATURE_COLS].values, features_array])

test_preds = np.zeros((len(testi), len(target_columns)))

for i, target in enumerate(target_columns):
    print(f'Predicting {target} with model {models[target]}')
    test_preds[:, i] = models[target].predict(X_combined_test)
     

Predicting X4_mean with model XGBRegressor(alpha=4.8413489204266845e-06, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=4.45328572879446e-06,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
             max_leaves=None, min_child_weight=56, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, ...)
Predicting X11_mean with model XGBRegressor(alpha=8.616392369610164e-08, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.3,

In [10]:
target_columns = ['X4', 'X11', 'X18', 'X50', 'X26', 'X3112']

test_df_copy = test_df.copy()
submission_df = test_df_copy[['id']].copy()
submission_df[target_columns] = test_preds

NameError: name 'test_df' is not defined

In [ ]:
submission_df.describe()

In [ ]:
train_df[target_columns].describe()

In [ ]:
submission_df.head()

In [ ]:
submission_df.to_csv('./data/submission.csv', index=False)